# Desafio 2 - Ciência e Governança de Dados
## Universidade Federal de Lavras  

### Autor: **Paulo Henrique dos Anjos Silveira** - 202310533  

---

# Modelos Preditivos de Machine Learning

## Objetivo
O presente trabalho tem como objetivo desenvolver modelos robustos e interpretáveis para prever impactos socioeconômicos nos municípios da Região Sudeste do Brasil baseados em agentes e fenômenos identificados na análise exploratória. Conforme proposto no Desafio II – Ciência e Governança de Dados da Zetta Lab/UFLA.

### Seleção de Modelos
Para esta tarefa, selecionamos 3 abordagens complementares:

1. **Random Forest**
2. **Gradient Boosting (XGBoost)**
3. **Ridge Regression**

In [90]:
# Importações para ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

## 1. Preparação dos Dados para Modelagem
### Importando biblitecas necessárias

In [91]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import randint, uniform

### Recupera o DataFrame salvo em CSV


In [92]:
csv_path = r"c:\Users\paulo\Downloads\Desafio-1-Ciencia-e-Governanca-de-Dados\dataFrame.csv"
if os.path.exists(csv_path):
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"Carregado {len(df):,} linhas em `df` a partir de: {csv_path}")
    except Exception as e:
        print("Erro ao ler CSV:", e)
else:
    print(f"Arquivo não encontrado: {csv_path}. Verifique o caminho ou gere o CSV no outro notebook.")

Carregado 1,661 linhas em `df` a partir de: c:\Users\paulo\Downloads\Desafio-1-Ciencia-e-Governanca-de-Dados\dataFrame.csv


### Mudanças no dataset e justificativa

- Alterações feitas no dataset:
  - Adicionadas colunas: `pib_per_capita`, `log_pib_pc`.
  - `populacao` foi mantida no DataFrame, mas removida da lista de preditores.
  - Substituímos `pib` por `log_pib_pc` na lista `variaveis_preditoras`.

- Justificativa:
  - O PIB bruto apresenta escala muito maior e alta correlação com `populacao`.
  - Usar PIB per capita (e aplicar log1p) melhora interpretabilidade e reduz
    multicolinearidade, tornando os coeficientes mais estáveis e coerentes.

In [93]:
try:
    df['pib_per_capita'] = df['pib'] / df['populacao']
    df['log_pib_pc'] = np.log1p(df['pib_per_capita'])
    print("Transformações aplicadas: 'pib_per_capita', 'log_pib_pc'")
except Exception as e:
    print("Erro ao criar transformações de PIB (verifique colunas 'pib' e 'populacao'):", e)

Transformações aplicadas: 'pib_per_capita', 'log_pib_pc'


### Função para preparar dados para modelagem

#### Por que utlizei o IDHM como impacto socioeconômico principal?

O Índice de Desenvolvimento Humano Municipal (IDHM) foi escolhido como a variável de impacto
socioeconômico principal por várias razões práticas e conceituais:

- **Multidimensionalidade:** o IDHM combina componentes de renda, educação e longevidade, oferecendo
  uma medida composta do bem-estar e desenvolvimento humano que captura efeitos econômicos e sociais.
- **Comparabilidade espacial:** o IDHM é calculado e disponível consistentemente para todos os municípios,
  permitindo comparações entre localidades e análises em escala regional.
- **Relevância de política pública:** como índice amplamente utilizado em estudos e políticas, resultados
  sobre fatores que influenciam o IDHM são diretamente acionáveis para gestores e formuladores de políticas.
- **Robustez e interpretação:** por ser uma métrica agregada, reduz ruído de indicadores pontuais isolados
  (ex.: apenas renda ou apenas expectativa de vida) e facilita a modelagem de impacto socioeconômico.

Limitações e observações:
- O IDHM é uma medida agregada que pode incorporar atrasos temporais (efeitos que aparecem somente depois de
  anos). Isso significa que relações observadas representam associações contemporâneas, não causalidade imediata.

In [94]:
def preparar_dados_ml(df, variavel_alvo, variaveis_preditoras):
    """
    Prepara dados para treinamento de modelos ML
    """
    X = df[variaveis_preditoras]
    y = df[variavel_alvo]
    
    # Verifica variância das features
    variancias = X.var()
    print(f"\n  Análise das Features:")
    print(f"   Total de amostras: {len(X)}")
    print(f"   Features com variância > 0: {(variancias > 0).sum()}/{len(variancias)}")
    print(f"   Features: {list(X.columns)}")
    
    return X, y, df

# Usando IDHM como variável alvo
print("Predição de IDHM (Desenvolvimento Humano Municipal)")

variaveis_preditoras = ['log_pib_pc', 'desemprego', 'internet', 
                        'homicidios', 'analfabetismo']
variavel_alvo = 'idhm'

X, y, dados = preparar_dados_ml(df, variavel_alvo, variaveis_preditoras)

print(f"\n Dados prontos para modelagem")
print(f"   Variável alvo: {variavel_alvo}")
print(f"   IDHM - Min: {y.min():.3f}, Max: {y.max():.3f}, Média: {y.mean():.3f}")

Predição de IDHM (Desenvolvimento Humano Municipal)

  Análise das Features:
   Total de amostras: 1661
   Features com variância > 0: 5/5
   Features: ['log_pib_pc', 'desemprego', 'internet', 'homicidios', 'analfabetismo']

 Dados prontos para modelagem
   Variável alvo: idhm
   IDHM - Min: 0.762, Max: 0.806, Média: 0.786


### Hiperparâmetros

In [95]:
param_dist_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None]
}

param_dist_xgb = {
    'n_estimators': randint(100,300),
    'max_depth': randint(3,10),
    'learning_rate': uniform(0.01,0.3),
    'subsample': uniform(0.6,0.4),
    'colsample_bytree': uniform(0.6,0.4)
}

param_dist_ridge = {'alpha': uniform(0.001, 10)}

## 2. Divisão em Treino/Teste e Normalização

In [96]:
# Divisão treino/teste com estratificação por estado
X_train, X_test, y_train, y_test, estados_train, estados_test = train_test_split(
    X, y, dados['sigla'], test_size=0.2, random_state=42
)

# Normalização das features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(" DIVISÃO DOS DADOS")

print(f"Treino: {len(X_train)} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"Teste:  {len(X_test)} amostras ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nFeatures normalizadas")
print(f"Média após normalização: {X_train_scaled.mean(axis=0).mean():.6f}")
print(f"Desvio padrão após normalização: {X_train_scaled.std(axis=0).mean():.6f}")

 DIVISÃO DOS DADOS
Treino: 1328 amostras (80.0%)
Teste:  333 amostras (20.0%)

Features normalizadas
Média após normalização: 0.000000
Desvio padrão após normalização: 1.000000


## 3. Modelo 1: Random Forest Regressor (Não-linear)

### Fórmula Geral
$$\hat{y}_{RF} = \frac{1}{B} \sum_{b=1}^{B} T_b(x)$$

Onde:
- $B$ = número de árvores (200)
- $T_b(x)$ = predição da árvore $b$
- $\hat{y}_{RF}$ = média das predições

### Feature Importance (Mean Decrease Impurity)
$$\text{Importance}_j = \frac{1}{B} \sum_{b=1}^{B} \sum_{t \in T_b} \mathbb{1}[\text{split}_t = j] \times \Delta \text{Impurity}_t$$

Onde:
- $j$ = feature de interesse
- $\Delta \text{Impurity}_t$ = redução de impureza no split $t$

In [97]:
# RandomizedSearchCV para Random Forest
rf_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist_rf,
    n_iter=20,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
rf_search.fit(X_train, y_train)
rf_model = rf_search.best_estimator_
print("Melhores parâmetros (RF):", rf_search.best_params_)

# Predições
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Métricas
r2_train_rf = r2_score(y_train, y_train_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
mae_test_rf = mean_absolute_error(y_test, y_test_pred_rf)

# Validação Cruzada
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_rf:.4f}")
print(f"   R² Teste:  {r2_test_rf:.4f}")
print(f"   RMSE:      {rmse_test_rf:.4f}")
print(f"   MAE:       {mae_test_rf:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

# Feature Importance
feature_importance_rf = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n Feature Importance (Random Forest):")
for idx, row in feature_importance_rf.iterrows():
    pct = row['Importance'] * 100
    print(f"   {row['Feature']:20s}: {pct:6.2f}%")

# Visualização de Feature Importance
fig_rf_imp = px.bar(
    feature_importance_rf,
    x='Importance',
    y='Feature',
    orientation='h',
    title='Random Forest: Importância das Features',
    labels={'Importance': 'Importância Relativa', 'Feature': 'Variável'},
    color='Importance',
    color_continuous_scale='Viridis'
)
fig_rf_imp.update_layout(showlegend=False, height=400)
fig_rf_imp.show()


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros (RF): {'max_depth': 19, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 269}

 Desempenho:
   R² Treino: 1.0000
   R² Teste:  1.0000
   RMSE:      0.0000
   MAE:       0.0000

 Validação Cruzada (5-fold):
   R² Médio: 1.0000 (+/- 0.0000)

 Feature Importance (Random Forest):
   desemprego          : 100.00%
   log_pib_pc          :   0.00%
   internet            :   0.00%
   analfabetismo       :   0.00%
   homicidios          :   0.00%


In [98]:
# Análise de resíduos do Random Forest
residuos_rf = y_test - y_test_pred_rf

fig_rf_resid = go.Figure()
fig_rf_resid.add_trace(go.Scatter(
    x=y_test_pred_rf,
    y=residuos_rf,
    mode='markers',
    marker=dict(size=8, color='rgba(31, 119, 180, 0.6)'),
    name='Resíduos'
))
fig_rf_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_rf_resid.update_layout(
    title='Random Forest: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_rf_resid.show()

## 4. Modelo 2: XGBoost (Gradient Boosting)

### Objetivo de Otimização
$$\text{Loss} = \sum_{i=1}^{n} l(y_i, \hat{y}_i) + \sum_{k=1}^{K} \Omega(f_k)$$

Onde:
- $l()$ = função de perda (squared error para regressão)
- $\Omega(f)$ = termo de regularização (penaliza complexidade)

### Predição Iterativa
$$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + \eta \cdot f_t(x_i)$$

Onde:
- $t$ = iteração (1 a 150)
- $\eta$ = learning rate (0.1)
- $f_t()$ = árvore treinada na iteração $t$

### Regularização
$$\Omega(f) = \gamma T + \frac{1}{2}\lambda \|w\|^2$$

Onde:
- $T$ = número de folhas
- $\gamma$ = penalidade por complexidade
- $\lambda$ = penalidade L2 dos pesos

In [99]:
xgb_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(random_state=42, verbosity=0, tree_method='hist'),
    param_distributions=param_dist_xgb,
    n_iter=30,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

xgb_search.fit(X_train, y_train)
xgb_model = xgb_search.best_estimator_
print('Melhores parâmetros (XGB):', xgb_search.best_params_)

# Predições
y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

# Métricas
r2_train_xgb = r2_score(y_train, y_train_pred_xgb)
r2_test_xgb = r2_score(y_test, y_test_pred_xgb)
rmse_test_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
mae_test_xgb = mean_absolute_error(y_test, y_test_pred_xgb)

# Validação Cruzada
cv_scores_xgb = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_xgb:.4f}")
print(f"   R² Teste:  {r2_test_xgb:.4f}")
print(f"   RMSE:      {rmse_test_xgb:.4f}")
print(f"   MAE:       {mae_test_xgb:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std():.4f})")

# Feature Importance
feature_importance_xgb = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n Feature Importance (XGBoost):")
for idx, row in feature_importance_xgb.iterrows():
    pct = row['Importance'] * 100
    print(f"   {row['Feature']:20s}: {pct:6.2f}%")

# Visualização
fig_xgb_imp = px.bar(
    feature_importance_xgb,
    x='Importance',
    y='Feature',
    orientation='h',
    title='XGBoost: Importância das Features (Ganho Médio)',
    labels={'Importance': 'Importância Relativa', 'Feature': 'Variável'},
    color='Importance',
    color_continuous_scale='Plasma'
)
fig_xgb_imp.update_layout(showlegend=False, height=400)
fig_xgb_imp.show()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhores parâmetros (XGB): {'colsample_bytree': np.float64(0.9332779646944658), 'learning_rate': np.float64(0.062009396052331626), 'max_depth': 3, 'n_estimators': 263, 'subsample': np.float64(0.672894435115225)}

 Desempenho:
   R² Treino: 0.9997
   R² Teste:  0.9994
   RMSE:      0.0004
   MAE:       0.0002

 Validação Cruzada (5-fold):
   R² Médio: 0.9991 (+/- 0.0002)

 Feature Importance (XGBoost):
   desemprego          :  86.27%
   analfabetismo       :   7.80%
   homicidios          :   2.62%
   internet            :   2.26%
   log_pib_pc          :   1.05%


In [100]:
# Análise de resíduos do XGBoost
residuos_xgb = y_test - y_test_pred_xgb

fig_xgb_resid = go.Figure()
fig_xgb_resid.add_trace(go.Scatter(
    x=y_test_pred_xgb,
    y=residuos_xgb,
    mode='markers',
    marker=dict(size=8, color='rgba(255, 127, 14, 0.6)'),
    name='Resíduos'
))
fig_xgb_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_xgb_resid.update_layout(
    title='XGBoost: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_xgb_resid.show()

## 5. Modelo 3: Ridge Regression

### Objetivo de Otimização (L2 Regularization)
$$J(\beta) = \sum_{i=1}^{n} (y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij})^2 + \alpha \sum_{j=1}^{p} \beta_j^2$$

Ou em forma matricial:
$$J(\beta) = (y - X\beta)^T(y - X\beta) + \alpha \beta^T \beta$$

### Solução Analítica
$$\hat{\beta}_{Ridge} = (X^T X + \alpha I)^{-1} X^T y$$

Onde:
- $\alpha$ = parâmetro de regularização (0.5 neste caso)
- $I$ = matriz identidade

### Propriedades Estatísticas
$$\mathbb{E}[\hat{\beta}_{Ridge}] \neq \beta \quad \text{(enviesado)}$$
$$\text{Var}[\hat{\beta}_{Ridge}] < \text{Var}[\hat{\beta}_{OLS}] \quad \text{(variância menor)}$$

### Interpretação dos Coeficientes
Para dados **normalizados** (média 0, desvio padrão 1):

Aumento de 1 desvio padrão em $x_j$ → aumento de $\hat{\beta}_j$ em $y$

$$\Delta y = \hat{\beta}_j \times \Delta x_j$$

In [101]:
# Treinamento do Ridge
ridge_search = RandomizedSearchCV(
    estimator=Ridge(), 
    param_distributions=param_dist_ridge, 
    n_iter=30, 
    cv=5, 
    scoring='r2', 
    random_state=42, 
    n_jobs=-1
)
ridge_search.fit(X_train_scaled, y_train)
ridge_model = ridge_search.best_estimator_
print('Best Ridge params:', ridge_search.best_params_)

# Predições
y_train_pred_ridge = ridge_model.predict(X_train_scaled)
y_test_pred_ridge = ridge_model.predict(X_test_scaled)

# Métricas
r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
mae_test_ridge = mean_absolute_error(y_test, y_test_pred_ridge)

# Validação Cruzada
cv_scores_ridge = cross_val_score(ridge_model, X_train_scaled, y_train, cv=5, scoring='r2')

print(f"\n Desempenho:")
print(f"   R² Treino: {r2_train_ridge:.4f}")
print(f"   R² Teste:  {r2_test_ridge:.4f}")
print(f"   RMSE:      {rmse_test_ridge:.4f}")
print(f"   MAE:       {mae_test_ridge:.4f}")
print(f"\n Validação Cruzada (5-fold):")
print(f"   R² Médio: {cv_scores_ridge.mean():.4f} (+/- {cv_scores_ridge.std():.4f})")

# Coeficientes
coeficientes_ridge = pd.DataFrame({
    'Feature': variaveis_preditoras,
    'Coeficiente': ridge_model.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)

print(f"\n Coeficientes (dados normalizados):")
print(f"   Intercepto: {ridge_model.intercept_:.6f}")
print(f"\n   Variável                Coeficiente")
print(f"   " + "-" * 40)
for idx, row in coeficientes_ridge.iterrows():
    coef = row['Coeficiente']
    print(f"   {row['Feature']:20s}  {coef:+10.6f}")

# Visualização dos coeficientes
fig_ridge_coef = px.bar(
    coeficientes_ridge,
    x='Coeficiente',
    y='Feature',
    orientation='h',
    title='Ridge Regression: Coeficientes Padronizados',
    labels={'Coeficiente': 'Valor do Coeficiente', 'Feature': 'Variável'},
    color='Coeficiente',
    color_continuous_scale='RdBu',
    color_continuous_midpoint=0
)
fig_ridge_coef.update_layout(showlegend=False, height=400)
fig_ridge_coef.show()

Best Ridge params: {'alpha': np.float64(9.700098521619942)}

 Desempenho:
   R² Treino: 0.2289
   R² Teste:  0.2518
   RMSE:      0.0138
   MAE:       0.0122

 Validação Cruzada (5-fold):
   R² Médio: 0.1827 (+/- 0.0588)

 Coeficientes (dados normalizados):
   Intercepto: 0.785769

   Variável                Coeficiente
   ----------------------------------------
   analfabetismo          -0.004993
   homicidios             -0.002472
   desemprego             +0.002470
   internet               +0.001883
   log_pib_pc             +0.000191


In [102]:
# Análise de resíduos do Ridge
residuos_ridge = y_test - y_test_pred_ridge

fig_ridge_resid = go.Figure()
fig_ridge_resid.add_trace(go.Scatter(
    x=y_test_pred_ridge,
    y=residuos_ridge,
    mode='markers',
    marker=dict(size=8, color='rgba(44, 160, 44, 0.6)'),
    name='Resíduos'
))
fig_ridge_resid.add_hline(y=0, line_dash='dash', line_color='red')
fig_ridge_resid.update_layout(
    title='Ridge Regression: Análise de Resíduos',
    xaxis_title='Valores Preditos',
    yaxis_title='Resíduos',
    height=400
)
fig_ridge_resid.show()

## 6. Comparação de Desempenho dos Modelos

In [103]:
# Tabela comparativa
comparacao_modelos = pd.DataFrame({
    'Modelo': ['Random Forest', 'XGBoost', 'Ridge Regression'],
    'R² Teste': [r2_test_rf, r2_test_xgb, r2_test_ridge],
    'R² CV (média)': [cv_scores_rf.mean(), cv_scores_xgb.mean(), cv_scores_ridge.mean()],
    'R² CV (std)': [cv_scores_rf.std(), cv_scores_xgb.std(), cv_scores_ridge.std()],
    'RMSE': [rmse_test_rf, rmse_test_xgb, rmse_test_ridge],
    'MAE': [mae_test_rf, mae_test_xgb, mae_test_ridge]
})

print("\n", comparacao_modelos.to_string(index=False))

# Visualização da comparação
fig_compare = go.Figure()

metricas = ['R² Teste', 'R² CV (média)', 'RMSE', 'MAE']
for i, modelo in enumerate(comparacao_modelos['Modelo']):
    valores = [
        comparacao_modelos.loc[i, 'R² Teste'],
        comparacao_modelos.loc[i, 'R² CV (média)'],
        comparacao_modelos.loc[i, 'RMSE'] / 0.3,  # Normalizar escala
        comparacao_modelos.loc[i, 'MAE'] / 0.3
    ]
    fig_compare.add_trace(go.Bar(
        name=modelo,
        x=metricas,
        y=valores
    ))

fig_compare.update_layout(
    title='Comparação de Desempenho dos Modelos',
    xaxis_title='Métrica',
    yaxis_title='Valor Normalizado',
    barmode='group',
    height=450
)
fig_compare.show()


           Modelo  R² Teste  R² CV (média)  R² CV (std)         RMSE          MAE
   Random Forest  1.000000       1.000000     0.000000 2.557337e-15 2.029408e-15
         XGBoost  0.999381       0.999124     0.000219 3.973723e-04 2.484006e-04
Ridge Regression  0.251799       0.182725     0.058801 1.381479e-02 1.223562e-02


In [104]:
# Scatter plot de valores reais vs preditos para todos os modelos
fig_predictions = go.Figure()

fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_rf,
    mode='markers',
    name='Random Forest',
    marker=dict(size=8, opacity=0.7)
))
fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_xgb,
    mode='markers',
    name='XGBoost',
    marker=dict(size=8, opacity=0.7)
))
fig_predictions.add_trace(go.Scatter(
    x=y_test, y=y_test_pred_ridge,
    mode='markers',
    name='Ridge',
    marker=dict(size=8, opacity=0.7)
))

# Linha perfeita
y_min, y_max = y_test.min(), y_test.max()
fig_predictions.add_trace(go.Scatter(
    x=[y_min, y_max],
    y=[y_min, y_max],
    mode='lines',
    name='Predição Perfeita',
    line=dict(dash='dash', color='black', width=2)
))

fig_predictions.update_layout(
    title='Comparação: Valores Reais vs Preditos',
    xaxis_title='Valores Reais (IDHM)',
    yaxis_title='Valores Preditos',
    height=500
)
fig_predictions.show()

## 7. Análise de Impacto das Variáveis Socioeconômicas

In [105]:
print("\nRanking de importância (média dos 3 modelos):")

importance = feature_importance_rf.copy()
importance.columns = ['Feature', 'RF_importance']

# Normalizar as importâncias para escala 0-1
rf_norm = feature_importance_rf.set_index('Feature')['Importance'].to_dict()
xgb_norm = feature_importance_xgb.set_index('Feature')['Importance'].to_dict()

# Criar ranking
ranking = []
for feat in variaveis_preditoras:
    rf_imp = rf_norm.get(feat, 0)
    xgb_imp = xgb_norm.get(feat, 0)
    ridge_coef = abs(coeficientes_ridge[coeficientes_ridge['Feature'] == feat]['Coeficiente'].values[0])
    ridge_coef_norm = ridge_coef / coeficientes_ridge['Coeficiente'].abs().max()
    
    media = (rf_imp + xgb_imp + ridge_coef_norm) / 3
    ranking.append({
        'Feature': feat,
        'RF': rf_imp,
        'XGB': xgb_imp,
        'Ridge': ridge_coef_norm,
        'Média': media
    })

ranking_df = pd.DataFrame(ranking).sort_values('Média', ascending=False)

for i, row in ranking_df.iterrows():
    pct = row['Média'] * 100
    print(f"   {i+1}. {row['Feature']:20s}: {pct:5.1f}%")

# Visualização
fig_consolidado = px.bar(
    ranking_df,
    x='Média',
    y='Feature',
    orientation='h',
    title='Importância das Variáveis (Random Forest, XGBoost e Ridge)',
    labels={'Média': 'Importância Relativa (0-1)', 'Feature': 'Variável Socioeconômica'},
    color='Média',
    color_continuous_scale='Turbo'
)
fig_consolidado.update_layout(showlegend=False, height=400)
fig_consolidado.show()


Ranking de importância (média dos 3 modelos):
   2. desemprego          :  78.6%
   5. analfabetismo       :  35.9%
   4. homicidios          :  17.4%
   3. internet            :  13.3%
   1. log_pib_pc          :   1.6%


# Conclusão

> O desenvolvimento humano sustentável exige políticas públicas que ultrapassem a lógica tradicional do crescimento econômico agregado e passem a priorizar a melhoria efetiva das condições de vida da população. Nesse sentido, focar exclusivamente no aumento do Produto Interno Bruto (PIB) pode ser insuficiente, especialmente quando esse crescimento se concentra em poucos setores ou grupos sociais. Torna-se, portanto, fundamental direcionar esforços para o aumento do PIB per capita e para a diversificação econômica, por meio de incentivos a pequenos negócios, fortalecimento de cadeias produtivas locais e estímulo ao empreendedorismo, garantindo uma distribuição mais equilibrada da renda.
>
> Paralelamente, políticas de geração de emprego devem ser pensadas de forma direcionada, considerando as desigualdades regionais e a existência de desemprego estrutural em determinados municípios. Programas que integrem qualificação profissional e incentivos fiscais à contratação local podem ampliar significativamente as oportunidades de inserção no mercado de trabalho, ao mesmo tempo em que fortalecem economias regionais fragilizadas.
>
> Outro eixo estratégico é a inclusão digital. A ampliação da infraestrutura de internet banda larga em municípios com baixa cobertura é essencial para reduzir desigualdades e ampliar o acesso a serviços, informação e oportunidades econômicas.
>
> A educação básica, em especial a alfabetização, permanece como um dos principais fatores de impacto no Índice de Desenvolvimento Humano Municipal (IDHM).
>
> Por fim, a integração entre educação, emprego, inclusão digital e segurança pública, aliada à testagem prévia das políticas em municípios-piloto, permite maior eficiência no uso dos recursos públicos e mais segurança na ampliação das iniciativas.

<div align="center">
    <img src="image.png" width="40%">
</div>